In [15]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import html5lib
import pandas as pd

def get_url(Departing, Arrival, Departure_date):
    TEMPLATES = "https://book-airpeace.crane.aero/ibe/availability?tripType=ONE_WAY&depPort={}&arrPort={}&departureDate={}%20%20%20%20%20%20%20%20&adult=1&child=0&infant=0&lang=en"
    url = TEMPLATES.format(Departing, Arrival, Departure_date)
    return url

# Prompt user for departure and arrival locations and departure date
a = input('''Choose your departure location from the following options:
ABJ>Abidjan, ABV>Abuja, ACC>Accra, AKR>Akure, ANA>Anambra, BJL>Banjul, BNI>Benincity, CBQ>Calabar, COO>Cotonou,
DSS>Dakar Blaise Diagne Internatio, DLA>Douala, ENU>Enugu, GMO>Gombe, ILR>Ilorin, JED>Jeddah, JNB>Johannesburg, 
KAD>Kaduna, KAN>Kano, LOS>Lagos, LFW>Lome, LGW>London, MIU>Maiduguri, MDI>Makurdi, ROB>Monrovia, BOM>Mumbai,
QOW>Owerri, PHC>Portharcourt, QUO>Uyo, QRW>Warri, YOL>Yola
Enter your departure location (e.g., ABV for Abuja): ''')

b = input('''Choose your arrival location from the following options:
ABJ>Abidjan, ABV>Abuja, ACC>Accra, AKR>Akure, ANA>Anambra, BJL>Banjul, BNI>Benincity, CBQ>Calabar, COO>Cotonou,
DSS>Dakar Blaise Diagne Internatio, DLA>Douala, ENU>Enugu, GMO>Gombe, ILR>Ilorin, JED>Jeddah, JNB>Johannesburg,
KAD>Kaduna, KAN>Kano, LOS>Lagos, LFW>Lome, LGW>London, MIU>Maiduguri, MDI>Makurdi, ROB>Monrovia, BOM>Mumbai,
QOW>Owerri, PHC>Portharcourt, QUO>Uyo, QRW>Warri, YOL>Yola
Enter your arrival location (e.g., LOS for Lagos): ''')

c = input('''DEPARTURE DATE
ENTER YOUR DEPARTURE DATE (e.g., dd.mm.yyyy): ''')

# Construct URL and print it
link = get_url(a, b, c)
print(link)

options = Options()
options.headless = True  # Run Chrome in headless mode

# Set up Chrome driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

driver.get(link)

sleep(2)

# Find flight rows
flight_rows = driver.find_elements(By.XPATH, '//div[@class="row w-100 no-gutters"]')
print(flight_rows)

# Initialize lists for storing flight details
depart_time_list = []
depart_loc_list = []
depart_date_list = []
flight_duration_list = []
total_stop_list = []
flight_no_list = []
arrival_time_list = []
arrival_loc_list = []
arrival_date_list = []
economy_price_list = []
executive_economy_price_list = []
business_price_list = []

# Parse the flight details
for WebElement in flight_rows:
    elementHTML = WebElement.get_attribute('outerHTML')
    elementsoup = BeautifulSoup(elementHTML, 'html.parser')

    # FLIGHT INFO BOX 1
    flight_box1 = elementsoup.find('div', {"class": "info-block"})
    depart_time = flight_box1.find("span", {"class": "time"})
    depart_loc = flight_box1.find("span", {"class": "port"})
    depart_date = flight_box1.find("span", {"class": "date"})
    depart_time_list.append(depart_time.text if depart_time else "N/A")
    depart_loc_list.append(depart_loc.text if depart_loc else "N/A")
    depart_date_list.append(depart_date.text if depart_date else "N/A")
    sleep(10)

    # FLIGHT INFO BOX 2
    flight_box2 = elementsoup.find('div', {"class": "info-row"})
    flight_duration = flight_box2.find("span", {"class": "flight-duration"})
    total_stop = flight_box2.find("span", {"class": "total-stop"})
    flight_no = flight_box2.find("span", {"class": "flight-no"})
    flight_duration_list.append(flight_duration.text if flight_duration else "N/A")
    total_stop_list.append(total_stop.text if total_stop else "N/A")
    flight_no_list.append(flight_no.text if flight_no else "N/A")
    sleep(10)

    # FLIGHT INFO BOX 3
    flight_box3 = elementsoup.find('div', {"class": "info-block text-right"})
    arrival_time = flight_box3.find("span", {"class": "time"})
    arrival_loc = flight_box3.find("span", {"class": "port"})
    arrival_date = flight_box3.find("span", {"class": "date"})
    arrival_time_list.append(arrival_time.text if arrival_time else "N/A")
    arrival_loc_list.append(arrival_loc.text if arrival_loc else "N/A")
    arrival_date_list.append(arrival_date.text if arrival_date else "N/A")
    sleep(10)

    # FLIGHT INFO BOX 4 (Money section)
    flight_box4 = elementsoup.find('div', {'class': "desktop-fare-block"})
    # Economy price
    economy_price = flight_box4.find("span", {"class": "currency-best-offer"})
    if economy_price is not None:
        economy_price_list.append(economy_price.text.strip())
    else:
        economy_price = flight_box4.find("span", {"class": "currency"})
        economy_price_list.append(economy_price.text.strip() if economy_price else "no seat")
    sleep(10)

    # Executive economy price
    executive_economy_price = flight_box4.find("span", {"class": "no-seat-text"})
    executive_economy_price_list.append(executive_economy_price.text.strip() if executive_economy_price else "no seat")

    # Business price
    business_price = flight_box4.find("span", {"class": "currency"})
    business_price_list.append(business_price.text.strip() if business_price else "no seat")
    sleep(10)

# Create a DataFrame with the collected data
Table = {
    'FLIGHT DEPART': depart_loc_list,
    'FLIGHT ARRIVAL': arrival_loc_list,
    'DEPART TIME': depart_time_list,
    'ARRIVAL TIME': arrival_time_list,
    'DEPART DATE': depart_date_list,
    'ARRIVAL DATE': arrival_date_list,
    'FLIGHT DURATION': flight_duration_list,
    'TOTAL STOP': total_stop_list,
    'FLIGHT NO': flight_no_list,
    'BUSINESS PRICE': business_price_list,
    'ECONOMY PRICE': economy_price_list,
    'EXECUTIVE ECONOMY PRICE': executive_economy_price_list
}
df = pd.DataFrame(Table)
print(df)


https://book-airpeace.crane.aero/ibe/availability?tripType=ONE_WAY&depPort=LOS&arrPort=ABV&departureDate=05.12.2024%20%20%20%20%20%20%20%20&adult=1&child=0&infant=0&lang=en
[<selenium.webdriver.remote.webelement.WebElement (session="d2cbfe5edcd9fe8faff68b095cb6352e", element="f.ED3574718DF0B05037F597F7D96B4FED.d.0276B6C1857003F68D0DD4E336C3B3C3.e.106")>, <selenium.webdriver.remote.webelement.WebElement (session="d2cbfe5edcd9fe8faff68b095cb6352e", element="f.ED3574718DF0B05037F597F7D96B4FED.d.0276B6C1857003F68D0DD4E336C3B3C3.e.107")>, <selenium.webdriver.remote.webelement.WebElement (session="d2cbfe5edcd9fe8faff68b095cb6352e", element="f.ED3574718DF0B05037F597F7D96B4FED.d.0276B6C1857003F68D0DD4E336C3B3C3.e.108")>, <selenium.webdriver.remote.webelement.WebElement (session="d2cbfe5edcd9fe8faff68b095cb6352e", element="f.ED3574718DF0B05037F597F7D96B4FED.d.0276B6C1857003F68D0DD4E336C3B3C3.e.109")>, <selenium.webdriver.remote.webelement.WebElement (session="d2cbfe5edcd9fe8faff68b095cb6352e", 

In [16]:
df

,FLIGHT DEPART,FLIGHT ARRIVAL,DEPART TIME,ARRIVAL TIME,DEPART DATE,ARRIVAL DATE,FLIGHT DURATION,TOTAL STOP,FLIGHT NO,BUSINESS PRICE,ECONOMY PRICE,EXECUTIVE ECONOMY PRICE
0,Lagos (LOS),Abuja (ABV),06:30,07:50,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7120,$ 139.60,$ 55.60,No Seat
1,Lagos (LOS),Abuja (ABV),08:00,09:20,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7428,no seat,$ 55.60,No Seat
2,Lagos (LOS),Abuja (ABV),10:35,11:55,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7122,$ 139.60,$ 55.60,No Seat
3,Lagos (LOS),Abuja (ABV),12:00,13:20,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7124,$ 67.60,$ 67.60,No Seat
4,Lagos (LOS),Abuja (ABV),13:55,15:05,05 Dec 2024,05 Dec 2024,1h 10m,Nonstop,P4-7126,$ 67.60,$ 67.60,No Seat
5,Lagos (LOS),Abuja (ABV),15:45,17:00,05 Dec 2024,05 Dec 2024,1h 15m,Nonstop,P4-7132,$ 67.60,$ 67.60,No Seat
6,Lagos (LOS),Abuja (ABV),17:45,19:05,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7128,no seat,$ 55.60,No Seat
7,Lagos (LOS),Abuja (ABV),19:00,20:20,05 Dec 2024,05 Dec 2024,1h 20m,Nonstop,P4-7130,$ 139.60,$ 55.60,No Seat


In [174]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import html5lib
import pandas as pd

def get_green_africa_url(Departing, Arrival, Departure_date):
    TEMPLATE = "https://greenafrica.com/booking/select?origin={}&destination={}&departure={}&adt=1&chd=0&inf=0"
    url = TEMPLATE.format(Departing, Arrival, Departure_date)
    return url

# Prompt user for departure and arrival locations and departure date
a = input('''Choose your departure location from the following options:
ABJ>Abidjan, ABV>Abuja, ACC>Accra, AKR>Akure, ANA>Anambra, BJL>Banjul, BNI>Benincity, CBQ>Calabar, COO>Cotonou,
DSS>Dakar Blaise Diagne Internatio, DLA>Douala, ENU>Enugu, GMO>Gombe, ILR>Ilorin, JED>Jeddah, JNB>Johannesburg, 
KAD>Kaduna, KAN>Kano, LOS>Lagos, LFW>Lome, LGW>London, MIU>Maiduguri, MDI>Makurdi, ROB>Monrovia, BOM>Mumbai,
QOW>Owerri, PHC>Portharcourt, QUO>Uyo, QRW>Warri, YOL>Yola
Enter your departure location (e.g., LOS for Lagos): ''')

b = input('''Choose your arrival location from the following options:
ABJ>Abidjan, ABV>Abuja, ACC>Accra, AKR>Akure, ANA>Anambra, BJL>Banjul, BNI>Benincity, CBQ>Calabar, COO>Cotonou,
DSS>Dakar Blaise Diagne Internatio, DLA>Douala, ENU>Enugu, GMO>Gombe, ILR>Ilorin, JED>Jeddah, JNB>Johannesburg,
KAD>Kaduna, KAN>Kano, LOS>Lagos, LFW>Lome, LGW>London, MIU>Maiduguri, MDI>Makurdi, ROB>Monrovia, BOM>Mumbai,
QOW>Owerri, PHC>Portharcourt, QUO>Uyo, QRW>Warri, YOL>Yola
Enter your arrival location (e.g., ABV for Abuja): ''')

c = input('''DEPARTURE DATE
Enter your departure date (e.g., yyyy-mm-dd): ''')

# Construct URL and print it
link = get_green_africa_url(a, b, c)
print(f"Constructed URL: {link}")

options = Options()
options.headless = True  # Run Chrome in headless mode

# Set up Chrome driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

driver.get(link)
sleep(20)
flight_rows = driver.find_elements(By.XPATH, '//div[@class="chakra-accordion css-1po3p4o"]')
print(flight_rows)

for WebElement in flight_rows:
    elementHTML = WebElement.get_attribute('outerHTML')
    elementsoup = BeautifulSoup(elementHTML, 'html.parser')

    # FLIGHT INFO BOX 1
    flight_box1 = elementsoup.find('div', {"class": "align-center w-full lg:w-max grow"})
    depart_time = flight_box1.find_all("h3", {"class": "text-h4 lg:text-[30px] font-700 text-[#17181A] text-center"})[0].text.strip()
    depart_loc = flight_box1.find_all("p", {"class": "text-sm lg:text-p mt-4 text-dark300 text-center"})[0].text.strip()
    duration = flight_box1.find("p", {"class": "text-12 lg:text-sm text-dark300 border-t-1 border-solid border-dark100 text-center pt-14 font-500"}).text.strip()
    arrive_time = flight_box1.find_all("h3", {"class": "text-h4 lg:text-[30px] font-700 text-[#17181A] text-center"})[1].text.strip()
    arrive_loc = flight_box1.find_all("p", {"class": "text-sm lg:text-p mt-4 text-dark300 text-center"})[1].text.strip()

    # FLIGHT INFO BOX 2
    flight_no = elementsoup.select_one('.align-center .lg\\:p-9:nth-of-type(1) p:nth-of-type(2)').text.strip()
    total_stop = elementsoup.select_one('.align-center .lg\\:p-9:nth-of-type(2) p:nth-of-type(2)').text.strip()

    
   

# Get all packages
    packages = elementsoup.find_all('div', class_='box-shadow')

    for package in packages:
        title = package.find('h4').get_text(strip=True)
        price = package.find('span', class_='text-[30px]').get_text(strip=True)
        details = package.find_all('div', class_='flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top')
        
        print(f"Package: {title}")
        print(f"Price: {price}")
        
        for detail in details:
            description = detail.find('p', class_='text-sm').get_text(strip=True)
            print(f"Detail: {description}")
        
        print("\n" + "-"*40 + "\n")

    print(f"Departure Time: {depart_time}")
    print(f"Departure Location: {depart_loc}")
    print(f"Duration: {duration}")
    print(f"Arrival Time: {arrive_time}")
    print(f"Arrival Location: {arrive_loc}")
    print(f"Flight Number: {flight_no}")
    print(f"Total Time: {total_stop}")



Constructed URL: https://greenafrica.com/booking/select?origin=LOS&destination=ABV&departure=2024-12-09&adt=1&chd=0&inf=0
[<selenium.webdriver.remote.webelement.WebElement (session="c2ac4b5e8bc204fe7affa03826ef086d", element="f.03B4A2330CC591FDF1AE310BD28237FD.d.4A14B9F8CC3E69FF63F76D0589D9EA8A.e.2")>]
Package: gSaver
Price: ₦93,000

----------------------------------------

Package: gClassic
Price: ₦100,000

----------------------------------------

Package: gFlex
Price: ₦109,000

----------------------------------------

Departure Time: 20:20
Departure Location: LOS
Duration: Duration 1h 40mins
Arrival Time: 22:00
Arrival Location: ABV
Flight Number: Q9302
Total Time: Direct


In [176]:
from bs4 import BeautifulSoup

html = """   <div class="chakra-collapse" style="overflow: hidden; display: none; opacity: 0; height: 0px;"><div role="region" id="accordion-panel-:re:" aria-labelledby="accordion-button-:re:" class="chakra-accordion__panel css-1inji5u"><div class="lg:hidden mt-16"><div class="border-y-1 border-solid border-dark100 divide-y-1 divide-solid divide-dark100"><div class="flex h-70 "><div class="w-6 md:w-10 bg-primary"></div><div class="align-center px-10 justify-between grow"><h6 class="text-h6 w-91 rounded-4 text-center h-38 text-brand_blue bg-primary text-brand_blue">gSaver</h6><h4 class="text-h4 text-brand_blue font-600">₦93,000</h4><div><button class="border-brand_blue border-1 border-solid rounded-6 h-40 px-26 text-p text-brand_blue hover:bg-brand_blue hover:text-white focus:ring-2 focus:ring-opacity-30 relative focus:ring-brand_blue inline-flex items-center justify-center  font-700">Select</button></div></div></div><div class="flex h-70 "><div class="w-6 md:w-10 bg-dark300"></div><div class="align-center px-10 justify-between grow"><h6 class="text-h6 w-91 rounded-4 text-center h-38 text-brand_blue bg-dark300 text-white">gClassic</h6><h4 class="text-h4 text-brand_blue font-600">₦100,000</h4><div><button class="border-brand_blue border-1 border-solid rounded-6 h-40 px-26 text-p text-brand_blue hover:bg-brand_blue hover:text-white focus:ring-2 focus:ring-opacity-30 relative focus:ring-brand_blue inline-flex items-center justify-center  font-700">Select</button></div></div></div><div class="flex h-70 "><div class="w-6 md:w-10 bg-brand_blue"></div><div class="align-center px-10 justify-between grow"><h6 class="text-h6 w-91 rounded-4 text-center h-38 text-brand_blue bg-brand_blue text-white">gFlex</h6><h4 class="text-h4 text-brand_blue font-600">₦109,000</h4><div><button class="border-brand_blue border-1 border-solid rounded-6 h-40 px-26 text-p text-brand_blue hover:bg-brand_blue hover:text-white focus:ring-2 focus:ring-opacity-30 relative focus:ring-brand_blue inline-flex items-center justify-center  font-700">Select</button></div></div></div></div><div class="w-full flex-center mt-12"><button class=" inline-flex items-center font-700 justify-center relative "><p class="text-center underline text-brand_blue font-400">Compare Fares</p></button></div></div><div class="p-16 lg:p-22 pb-20 lg:pb-56 hidden lg:grid grid-cols-3 gap-40 lg:gap-24 xl:gap-40"><div class="basis-full lg:basis-[29%] l mb-7"><div class="box-shadow rounded-t-16 relative h-full flex flex-col  "><h4 class="text-h4 rounded-t-16 bg-primary text-center font-700 text-brand_blue p-16">gSaver</h4><div class="py-32 px-48 lg:px-28 xl:px-48 bgwhite flex-center flex-col gap-32"><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/smallbox.svg" alt="Must fit in the overhead locker or under your seat (40cm x 20cm x 55cm)"></figure></div><p class="text-md font-700 text-brand_blue text-center">7kg Hand Baggage</p><p class="text-sm text-center text-black">Must fit in the overhead locker or under your seat (40cm x 20cm x 55cm)</p></div><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/nocheckinbaggageicon.svg" alt="gSaver NOT ELIGIBLE FOR FREE CHECK-IN BAGGAGE"></figure></div><p class="text-md font-700 text-brand_blue text-center">Check-in Baggage</p><p class="text-sm text-center text-black">gSaver NOT ELIGIBLE FOR FREE CHECK-IN BAGGAGE</p></div><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/smallseat.svg" alt="Automatically Assigned (Pay for Preferred Seat)"></figure></div><p class="text-md font-700 text-brand_blue text-center">Seat Selection</p><p class="text-sm text-center text-black">Automatically Assigned (Pay for Preferred Seat)</p></div><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/bigbox.svg" alt="Airport check-in @ N2,000"></figure></div><p class="text-md font-700 text-brand_blue text-center">Airport Check-in</p><p class="text-sm text-center text-black">Airport check-in @ N2,000</p></div></div><div class="px-48 lg:px-28 xl:px-48 mb-32 mt-auto"><div><button class="border-brand_blue border-1 border-solid rounded-6 h-62 px-48 text-h6 text-brand_blue hover:bg-brand_blue hover:text-white focus:ring-2 focus:ring-opacity-30 relative focus:ring-brand_blue inline-flex items-center justify-center  font-700 w-full"><span class="text-[30px]">  ₦93,000</span></button></div></div><p class="text-p bg-primary text-center flex-center gap-6 text-brand_blue p-10"><svg width="16" height="16" fill="none" xmlns="http://www.w3.org/2000/svg"><g fill="#5045B8"><path d="M8.67 12.823c0 .448-.352.8-.8.8H6.174v1.248h.32c.544 0 .96.416.96.96h-3.84c0-.544.416-.96.96-.96h.32v-1.248H3.646c-.352 0-.64-.224-.768-.544-1.792-5.216-1.792-5.024-1.792-5.216 0-.352.192-.64.544-.768.416-.16.864.096 1.024.48 1.696 4.896 1.536 4.416 1.536 4.448h3.68c.416 0 .8.384.8.8Z"></path><path d="M5.09 5.271a1.276 1.276 0 0 0-1.664-.736A1.276 1.276 0 0 0 2.69 6.2l1.824 4.544c.416 1.024 1.504.8 1.6.8l3.072.032 2.495 3.84a.98.98 0 0 0 1.377.256 1 1 0 0 0 .223-1.344l-2.784-4.224a.986.986 0 0 0-.8-.448l-2.911-.032L5.09 5.271ZM2.782 4.246a1.696 1.696 0 1 0 0-3.392 1.696 1.696 0 0 0 0 3.392ZM11.838.502a3.073 3.073 0 0 0-3.072 3.072v2.432a3.073 3.073 0 0 0 6.144 0V3.574C14.942 1.91 13.534.502 11.838.502Zm2.464 5.536A2.43 2.43 0 0 1 11.87 8.47a2.43 2.43 0 0 1-2.432-2.432V3.606a2.411 2.411 0 0 1 2.4-2.464 2.43 2.43 0 0 1 2.432 2.432v2.464h.032Z"></path><path d="M13.663 6.037V3.605c0-.992-.832-1.824-1.824-1.824-.992 0-1.792.8-1.792 1.792v2.432c0 .992.8 1.792 1.792 1.792s1.824-.768 1.824-1.76Z"></path></g></svg><span class="font-700">1</span> seat(s) left</p></div></div><div class="basis-full lg:basis-[29%] l mb-7"><div class="box-shadow rounded-t-16 relative h-full flex flex-col  "><h4 class="text-h4 rounded-t-16 bg-dark300 text-center font-700 text-white p-16">gClassic</h4><div class="py-32 px-48 lg:px-28 xl:px-48 bgwhite flex-center flex-col gap-32"><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/smallbox.svg" alt="Must fit in the overhead locker or under your seat (40cm x 20cm x 55cm)"></figure></div><p class="text-md font-700 text-brand_blue text-center">7kg Hand Baggage</p><p class="text-sm text-center text-black">Must fit in the overhead locker or under your seat (40cm x 20cm x 55cm)</p></div><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/bigbox.svg" alt="1 item, not weighing more than 15kg"></figure></div><p class="text-md font-700 text-brand_blue text-center">Check-in Baggage</p><p class="text-sm text-center text-black">1 item, not weighing more than 15kg</p></div><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/smallseat.svg" alt="Automatically Assigned (Pay for Preferred Seat)"></figure></div><p class="text-md font-700 text-brand_blue text-center">Seat Selection</p><p class="text-sm text-center text-black">Automatically Assigned (Pay for Preferred Seat)</p></div><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/airportcheckin.svg" alt="Free"></figure></div><p class="text-md font-700 text-brand_blue text-center">Airport Check-in</p><p class="text-sm text-center text-black">Free</p></div></div><div class="px-48 lg:px-28 xl:px-48 mb-32 mt-auto"><div><button class="border-brand_blue border-1 border-solid rounded-6 h-62 px-48 text-h6 text-brand_blue hover:bg-brand_blue hover:text-white focus:ring-2 focus:ring-opacity-30 relative focus:ring-brand_blue inline-flex items-center justify-center  font-700 w-full"><span class="text-[30px]">  ₦100,000</span></button></div></div><p class="text-p bg-dark300 text-center flex-center gap-6 text-white p-10"><svg width="21" height="21" fill="none" xmlns="http://www.w3.org/2000/svg"><g clip-path="url(#seats_left_white_svg__a)" fill="#fff"><path d="M10.886 16.818c0 .56-.44 1-1 1h-2.12v1.56h.4c.68 0 1.2.52 1.2 1.2h-4.8c0-.68.52-1.2 1.2-1.2h.4v-1.56h-1.56c-.44 0-.8-.28-.96-.68-2.24-6.52-2.24-6.28-2.24-6.52 0-.44.24-.8.68-.96.52-.2 1.08.12 1.28.6 2.12 6.12 1.92 5.52 1.92 5.56h4.6c.52 0 1 .48 1 1Z"></path><path d="M6.407 7.38a1.595 1.595 0 0 0-2.08-.92c-.84.32-1.24 1.24-.92 2.08l2.28 5.68c.52 1.28 1.88 1 2 1l3.84.04 3.12 4.8c.36.56 1.16.72 1.72.32.52-.4.64-1.12.28-1.68l-3.48-5.28c-.2-.32-.6-.56-1-.56l-3.64-.04-2.12-5.44ZM3.526 6.1a2.12 2.12 0 1 0 0-4.24 2.12 2.12 0 0 0 0 4.24ZM14.848 1.42c-2.12 0-3.84 1.72-3.84 3.84V8.3a3.841 3.841 0 0 0 7.68 0V5.26c.04-2.08-1.72-3.84-3.84-3.84Zm3.08 6.92c0 1.68-1.36 3.04-3.04 3.04s-3.04-1.36-3.04-3.04V5.3a3.014 3.014 0 0 1 3-3.08c1.68 0 3.04 1.36 3.04 3.04v3.08h.04Z"></path><path d="M17.13 8.34V5.3c0-1.24-1.04-2.28-2.28-2.28-1.24 0-2.24 1-2.24 2.24V8.3c0 1.24 1 2.24 2.24 2.24 1.24 0 2.28-.96 2.28-2.2Z"></path></g><defs><clipPath id="seats_left_white_svg__a"><path fill="#fff" transform="translate(.047 .791)" d="M0 0h20v20H0z"></path></clipPath></defs></svg><span class="font-700">1</span> seat(s) left</p></div></div><div class="basis-full lg:basis-[29%] l mb-7"><div class="box-shadow rounded-t-16 relative h-full flex flex-col  "><h4 class="text-h4 rounded-t-16 bg-brand_blue text-center font-700 text-white p-16">gFlex</h4><div class="py-32 px-48 lg:px-28 xl:px-48 bgwhite flex-center flex-col gap-32"><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/smallbox.svg" alt="Must fit in the overhead locker or under your seat (40cm x 20cm x 55cm)"></figure></div><p class="text-md font-700 text-brand_blue text-center">7kg Hand Baggage</p><p class="text-sm text-center text-black">Must fit in the overhead locker or under your seat (40cm x 20cm x 55cm)</p></div><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/bigbox.svg" alt="1 item, not weighing more than 23kg"></figure></div><p class="text-md font-700 text-brand_blue text-center">Check-in Baggage</p><p class="text-sm text-center text-black">1 item, not weighing more than 23kg</p></div><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/smallseat.svg" alt="Free"></figure></div><p class="text-md font-700 text-brand_blue text-center">Seat Selection</p><p class="text-sm text-center text-black">Free</p></div><div class="flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top"><div class="mx-auto"><figure><img src="https://gaassetstorage.s3.eu-west-2.amazonaws.com/svgassets/airportcheckin.svg" alt="Free"></figure></div><p class="text-md font-700 text-brand_blue text-center">Airport Check-in</p><p class="text-sm text-center text-black">Free</p></div></div><div class="px-48 lg:px-28 xl:px-48 mb-32 mt-auto"><div><button class="border-brand_blue border-1 border-solid rounded-6 h-62 px-48 text-h6 text-brand_blue hover:bg-brand_blue hover:text-white focus:ring-2 focus:ring-opacity-30 relative focus:ring-brand_blue inline-flex items-center justify-center  font-700 w-full"><span class="text-[30px]">  ₦109,000</span></button></div></div><p class="text-p bg-brand_blue text-center flex-center gap-6 text-white p-10"><svg width="21" height="21" fill="none" xmlns="http://www.w3.org/2000/svg"><g clip-path="url(#seats_left_white_svg__a)" fill="#fff"><path d="M10.886 16.818c0 .56-.44 1-1 1h-2.12v1.56h.4c.68 0 1.2.52 1.2 1.2h-4.8c0-.68.52-1.2 1.2-1.2h.4v-1.56h-1.56c-.44 0-.8-.28-.96-.68-2.24-6.52-2.24-6.28-2.24-6.52 0-.44.24-.8.68-.96.52-.2 1.08.12 1.28.6 2.12 6.12 1.92 5.52 1.92 5.56h4.6c.52 0 1 .48 1 1Z"></path><path d="M6.407 7.38a1.595 1.595 0 0 0-2.08-.92c-.84.32-1.24 1.24-.92 2.08l2.28 5.68c.52 1.28 1.88 1 2 1l3.84.04 3.12 4.8c.36.56 1.16.72 1.72.32.52-.4.64-1.12.28-1.68l-3.48-5.28c-.2-.32-.6-.56-1-.56l-3.64-.04-2.12-5.44ZM3.526 6.1a2.12 2.12 0 1 0 0-4.24 2.12 2.12 0 0 0 0 4.24ZM14.848 1.42c-2.12 0-3.84 1.72-3.84 3.84V8.3a3.841 3.841 0 0 0 7.68 0V5.26c.04-2.08-1.72-3.84-3.84-3.84Zm3.08 6.92c0 1.68-1.36 3.04-3.04 3.04s-3.04-1.36-3.04-3.04V5.3a3.014 3.014 0 0 1 3-3.08c1.68 0 3.04 1.36 3.04 3.04v3.08h.04Z"></path><path d="M17.13 8.34V5.3c0-1.24-1.04-2.28-2.28-2.28-1.24 0-2.24 1-2.24 2.24V8.3c0 1.24 1 2.24 2.24 2.24 1.24 0 2.28-.96 2.28-2.2Z"></path></g><defs><clipPath id="seats_left_white_svg__a"><path fill="#fff" transform="translate(.047 .791)" d="M0 0h20v20H0z"></path></clipPath></defs></svg><span class="font-700">1</span> seat(s) left</p></div></div></div></div></div>
"""

soup = BeautifulSoup(html, 'html.parser')

# Get all packages
packages = soup.find_all('div', class_='box-shadow')

for package in packages:
    title = package.find('h4').get_text(strip=True)
    price = package.find('span', class_='text-[30px]').get_text(strip=True)
    details = package.find_all('div', class_='flex text-center flex-col gap-8 px-10 lg:px-0 xl:px-10 min-h-[150px] align-top')
    
    print(f"Package: {title}")
    print(f"Price: {price}")
    
    for detail in details:
        description = detail.find('p', class_='text-sm').get_text(strip=True)
        print(f"Detail: {description}")
    
    print("\n" + "-"*40 + "\n")

# Optionally, you can refine the code to extract specific data points as needed


Package: gSaver
Price: ₦93,000
Detail: Must fit in the overhead locker or under your seat (40cm x 20cm x 55cm)
Detail: gSaver NOT ELIGIBLE FOR FREE CHECK-IN BAGGAGE
Detail: Automatically Assigned (Pay for Preferred Seat)
Detail: Airport check-in @ N2,000

----------------------------------------

Package: gClassic
Price: ₦100,000
Detail: Must fit in the overhead locker or under your seat (40cm x 20cm x 55cm)
Detail: 1 item, not weighing more than 15kg
Detail: Automatically Assigned (Pay for Preferred Seat)
Detail: Free

----------------------------------------

Package: gFlex
Price: ₦109,000
Detail: Must fit in the overhead locker or under your seat (40cm x 20cm x 55cm)
Detail: 1 item, not weighing more than 23kg
Detail: Free
Detail: Free

----------------------------------------



# QATAR AIRWAYS


In [2]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

def get_qatar_airways_url(Departing, Arrival, Departure_date):
    TEMPLATE = "https://www.qatarairways.com/app/booking/flight-selection?widget=QR&searchType=F&addTaxToFare=Y&minPurTime=0&selLang=en&tripType=O&fromStation={}&toStation={}&departing={}&bookingClass=E&adults=1&children=0&infants=0&ofw=0&teenager=0&flexibleDate=off"
    url = TEMPLATE.format(Departing, Arrival, Departure_date)
    return url

# Prompt user for departure and arrival locations and departure date
a = input('''Choose your departure location from the following options:
DOH>Doha, JFK>New York, LHR>London, CDG>Paris, DXB>Dubai, HND>Tokyo, SIN>Singapore, SYD>Sydney
Enter your departure location (e.g., DOH for Doha): ''')

b = input('''Choose your arrival location from the following options:
DOH>Doha, JFK>New York, LHR>London, CDG>Paris, DXB>Dubai, HND>Tokyo, SIN>Singapore, SYD>Sydney
Enter your arrival location (e.g., JFK for New York): ''')

c = input('''DEPARTURE DATE
Enter your departure date (e.g., yyyy-mm-dd): ''')

# Construct URL and print it
link = get_qatar_airways_url(a, b, c)
print(f"Constructed URL: {link}")

options = Options()
options.headless = True  # Run Chrome in headless mode

# Set up Chrome driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

driver.get(link)
sleep(20)



Constructed URL: https://www.qatarairways.com/app/booking/flight-selection?widget=QR&searchType=F&addTaxToFare=Y&minPurTime=0&selLang=en&tripType=O&fromStation=IST&toStation=ABV&departing=2024-12-10&bookingClass=E&adults=1&children=0&infants=0&ofw=0&teenager=0&flexibleDate=off


In [4]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

flight_rows = driver.find_elements(By.XPATH, '//*[@id="at-flight-search-result-1"]/div/div')
print(flight_rows)



[<selenium.webdriver.remote.webelement.WebElement (session="62a424f8558214467de2abfc2b6a17b8", element="f.3E8FE5120F6CBE7F125604CEBC49FB9A.d.0288010D7513AE3725B14E6DFB11BC76.e.67")>]


In [ ]:
for WebElement in flight_rows:
    elementHTML = WebElement.get_attribute('outerHTML')
    elementsoup = BeautifulSoup(elementHTML, 'html.parser')

    # FLIGHT INFO BOX 1
    flight_box1 = elementsoup.find('div', {'_ngcontent-ng-c3579460230': True, 'class': 'flight-card'})
    depart_time = flight_box1.find("h3",{'_ngcontent-ng-c3579460230': True,"class": "mb-1 at-flight-card-depart-time"}).text.strip()
    arrive_time = flight_box1.find("span",{'_ngcontent-ng-c3579460230':True, "class":"at-flight-card-arrival-time"}).text.strip()
    depart_loc =  flight_box1.find("p",{'_ngcontent-ng-c3579460230':True, "class":"at-flight-card-origin-code"}).text.strip()
    arrive_loc =  flight_box1.find("p",{'_ngcontent-ng-c3579460230':True,"class":"text-right at-flight-card-destination-code"}).text.strip()
    duration = flight_box1.find("div",{'_ngcontent-ng-c1652489756':True, "class":"ng-star-inserted", 'tabindex': '0'}).text.strip()


    # FLIGHT INFO BOX 2 
    flight_box2 = elementsoup.find('div',{'_ngcontent-ng-c3064091962':True, "class":"ng-tns-c3064091962-21 col-6 ng-star-inserted", ""})
    # econmy_price = flight_box2.find('span',{'_ngcontent-ng-c3064091962':True, "class":"fit-child ng-tns-c3064091962-21" }).text.strip()
    # business_price  = flight_box2.find('span',{'_ngcontent-ng-c3064091962':True, "class":"fit-child ng-tns-c3064091962-21"}).text.strip()

    print(depart_time)
    print(arrive_time)
    print(depart_loc)
    print(arrive_loc)
    print(duration)  
    print(flight_box2)
    # print(econmy_price)
    # print(business_price)

20:10
07:20
SAW
ABV
1 Stop, 13h 10m
None


In [ ]:
<div _ngcontent-ng-c3064091962 role="button" class="ng-tns-c3064091962-21 col-6 ng-star-inserted" style>

In [41]:
from datetime import datetime

def reformat_date_for_links(unique_date_input: str) -> dict:
    """
    Reformats a single input date to two different formats for two links.
    
    Args:
        unique_date_input (str): Input date in 'YYYY-MM-DD' format.
    
    Returns:
        dict: A dictionary with dates in the required formats.
    """
    try:
        # Parse the input date (assuming the input is in 'YYYY-MM-DD' format)
        date_obj = datetime.strptime(unique_date_input, '%Y-%m-%d')
        
        # Convert to the two required formats
        link1_format = date_obj.strftime('%d.%m.%Y')  # Format for the first link
        link2_format = date_obj.strftime('%Y-%m-%d')  # Format for the second link
        
        # Return the formatted dates as a dictionary
        return {
            'link1_date': link1_format,
            'link2_date': link2_format
        }
    except ValueError:
        raise ValueError(f"Invalid input date: {unique_date_input}. Expected format is 'YYYY-MM-DD'.")

# Example usage
unique_date_input = '2024-10-3'  # A unique input date in 'YYYY-MM-DD'
formatted_dates = reformat_date_for_links(unique_date_input)

print(f"Date for Link 1 (dd.mm.yyyy): {formatted_dates['link1_date']}")
print(f"Date for Link 2 (yyyy-mm-dd): {formatted_dates['link2_date']}")


Date for Link 1 (dd.mm.yyyy): 03.10.2024
Date for Link 2 (yyyy-mm-dd): 2024-10-03
